# PROJECT - BUS ROUTE FOR 16 CITIES

#### An origin-destination 16x16 matrix represents the distance between 16 districts in the capital city of San Jose, Costa Rica. The objective is to find an optimized route that connects all the districts with the minimal distance. This solution is intended to be used as a new bus route that is needed in the east zone of the city.
#### The task is modeled as a traveling salesman problem to be solved with: 1. PuLP, a linear programming method from Python. 2. Local search methods.

#### METHOD 1: LINEAR PROGRAMMING WITH PuLP

In [1]:
from pulp import *
import pandas as pd

In [2]:
dfobj = pd.read_csv('C:\\Users\\estef\\OneDrive\\Escritorio\\Estudios\\China\\Clases\\Fall 2022\\Optimization Research\\Assignments\\ODMatrix.csv', delimiter=';', header=None)
origin_destination_matrix = [list(row) for row in dfobj.values]
print(origin_destination_matrix)

[[0, 1, 3, 2, 4, 5, 6, 5, 9, 3, 3, 3, 8, 6, 7, 8], [1, 0, 2, 1, 5, 6, 5, 4, 8, 4, 4, 4, 9, 7, 8, 9], [3, 2, 0, 3, 7, 6, 7, 2, 6, 2, 2, 6, 11, 9, 10, 11], [2, 1, 3, 0, 4, 5, 6, 5, 9, 5, 5, 3, 8, 6, 7, 8], [4, 5, 7, 4, 0, 1, 10, 9, 13, 9, 9, 3, 8, 6, 7, 8], [5, 6, 6, 5, 1, 0, 10, 9, 12, 9, 8, 3, 8, 7, 7, 8], [6, 5, 7, 6, 10, 10, 0, 5, 9, 8, 9, 9, 14, 13, 13, 14], [5, 4, 2, 5, 9, 9, 5, 0, 4, 3, 4, 4, 10, 9, 9, 10], [9, 8, 6, 9, 13, 12, 9, 4, 0, 7, 8, 7, 13, 12, 12, 13], [3, 4, 2, 5, 9, 9, 8, 3, 7, 0, 3, 8, 13, 11, 12, 13], [3, 4, 2, 5, 9, 8, 9, 4, 8, 3, 0, 8, 13, 11, 12, 13], [3, 4, 6, 3, 3, 3, 9, 4, 7, 8, 8, 0, 5, 3, 4, 5], [8, 9, 11, 8, 8, 8, 14, 10, 13, 13, 13, 5, 0, 2, 9, 10], [6, 7, 9, 6, 6, 7, 13, 9, 12, 11, 11, 3, 2, 0, 7, 8], [7, 8, 10, 7, 7, 7, 13, 9, 12, 12, 12, 4, 9, 7, 0, 1], [8, 9, 11, 8, 8, 8, 14, 10, 13, 13, 13, 5, 10, 8, 1, 0]]


In [3]:
#Create variables with the parameters of the matrix to work with
cities = len(origin_destination_matrix)
indexes = range(cities)
print("Cities: ", cities)
print("Indexes: ", indexes)

Cities:  16
Indexes:  range(0, 16)


In [4]:
#Create a list of all the coordinates combinations of the matrix, excluding the center diagonal of 0's
E = [(i,j) for i in indexes for j in indexes if i!=j]

In [5]:
#Instruction for minimizing the problem
m = LpProblem("TSP",LpMinimize)

In [6]:
#Create a dictionary of all coordinate pairs
#Associate each pair with an X to assign 0 or 1 afterwards if the city will be chosen as part of the optimal route
x = LpVariable.dicts('x',E,cat=LpBinary)

In [7]:
#Create the objective function of costs per each pair
#The costs are the distances stated in the origin-destination matrix, excluding the diagonal
m += sum([origin_destination_matrix[i][j]*x[(i,j)]for (i,j) in E])

In [8]:
#Create the constraints
#It limits to a step size of 1, where from one origin, movement can only occur to one destination at a time
#The function calculates each pair both back and forth 
for i in indexes:
    m += sum([x[(i,j)] for j in indexes if i!=j]) == 1, "One out of "+str(i)
    m += sum([x[(j,i)] for j in indexes if i!=j]) == 1, "One in to "+str(i)

In [9]:
#Instructions for preliminar solution of the system without adding the restriction of subtours
m.solve()
vobj = value(m.objective)
xsol = {e:value(x[e]) for e in E}
print("- Optimal routes without eliminating subtours:")
sol_summary={x for x,y in xsol.items() if y!=0}
print("Segments within route: ", sol_summary)

- Optimal routes without eliminating subtours:
Segments within route:  {(0, 1), (10, 9), (15, 14), (8, 7), (2, 10), (5, 4), (6, 8), (9, 2), (11, 3), (12, 13), (4, 5), (7, 6), (13, 12), (1, 0), (14, 15), (3, 11)}


In [10]:
#Find successive cities that have the shortest path between them
def subtour(xsol):
    succ = 0
    subt = [succ]
    while True:
        succ = sum(xsol[succ,j]*j for j in indexes if j!=succ)
        if succ == 0: break
        subt.append(int(succ))
    return subt

In [11]:
#Iterate the previous definition to find all possible short subtours
#The function will add more nodes as it runs out of shorter routes
while True:
    subt = subtour(xsol)
    if len(subt) == cities:
        print("- Optimal tour excluding subtours")
        print("Optimal tour found: %r"%subt)
        print("Optimal tour length: %g"%vobj)
        break
    print("- Identify subtours")
    print("Subtour found: %r"%subt)
    m += sum([x[(i,j)] for i in subt for j in subt if i!=j])<= len(subt)-1
    m.solve()
    vobj = value(m.objective)
    xsol = {e: value(x[e]) for e in E}

- Identify subtours
Subtour found: [0, 1]
- Identify subtours
Subtour found: [0, 11]
- Identify subtours
Subtour found: [0, 6, 1]
- Identify subtours
Subtour found: [0, 1, 3, 11]
- Identify subtours
Subtour found: [0, 11, 3, 6, 1]
- Identify subtours
Subtour found: [0, 6, 1, 3]
- Identify subtours
Subtour found: [0, 1, 3]
- Identify subtours
Subtour found: [0, 11, 8, 7, 6, 1, 3]
- Identify subtours
Subtour found: [0, 10]
- Identify subtours
Subtour found: [0, 9]
- Identify subtours
Subtour found: [0, 9, 2, 10]
- Identify subtours
Subtour found: [0, 10, 9, 2, 1, 3, 11]
- Identify subtours
Subtour found: [0, 10, 9, 2, 3, 11]
- Identify subtours
Subtour found: [0, 10, 9, 2, 7, 8, 6, 1, 3, 11]
- Identify subtours
Subtour found: [0, 4, 5, 11, 3, 1]
- Identify subtours
Subtour found: [0, 10, 9, 2, 3, 1]
- Identify subtours
Subtour found: [0, 11, 5, 4, 3, 1, 6]
- Identify subtours
Subtour found: [0, 10, 9, 2, 6, 3, 1]
- Identify subtours
Subtour found: [0, 10, 9, 2, 7, 8, 6]
- Identify subtou

#### METHOD 2: LOCAL SEARCH

In [13]:
#The vector is filled with zeros in increasing order
def initial_solution(length):
    solution = [0]*length
    i = 0
    while i < length:
        solution[i] = i
        i = i + 1
    return solution

In [14]:
#A new variable is created to store the most recent solution
#vector before its order is updated in the iteration process
def backup_solution(sol):
    backsol = [0]*len(sol)
    i = 0
    while i < len(sol):
        backsol[i] = sol[i]
        i = i + 1
    return backsol

In [15]:
#The function calculates the total length of the
#tour, adding the distances indicated in the origin-destination 
#matrix. This is the target number to optimize when iterating 
#the arrangement of the solution vector.
def tour_length(solution):
    ans = 0
    i = 0
    while i < len(solution):
        if i + 1 < len(solution):
            ans = ans + origin_destination_matrix[solution[i]][solution[i+1]]
        else:
            ans = ans + origin_destination_matrix[solution[i]][solution[0]]
        i = i + 1
    return ans

In [16]:
#The values in the solution vector move one place. For instance,
#The solution that was previously in place 1, will now be
#located in place 2.
def neighborhood(solution,location):
    backsol = backup_solution(solution)
    backindex = backsol[location]
    if location + 1 < len(solution):
        backsol[location] = backsol[location + 1]
        backsol[location + 1] = backindex
    else:
        backsol[location] = backsol[0]
        backsol[0] = backindex
    return backsol

In [17]:
#Create the variables that will store the final results
#of the optimal route and minimal distance.
size = len(origin_destination_matrix)
seq = initial_solution(size)
obj = tour_length(seq)
print("initial solution :",seq)
print("initial tour length: ",obj)

initial solution : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
initial tour length:  71


In [18]:
#Iteration for the optimal order of the nodes takes place.
#The total length is recalculated to determine if the solution 
#is improving. The number of iterations is reduced to a maximum
#of 5 considering the limitations of computational resources. 
iter = 1
while iter <= 5:
    print("-------------------------------------")
    print("iter: ",iter)
    localseq = backup_solution(seq)
    localobj = obj
    loc = 0
    while loc < size:
        newseq = neighborhood(seq,loc)
        newobj = tour_length(newseq)
        if newobj < localobj:
            localseq = newseq
            localobj = newobj
            print("updated solution: ",localseq)
            print("updated tour length: ",localobj)
        loc = loc + 1
    if localobj < obj:
        seq = backup_solution(localseq)
        obj = localobj
    else:
        print("no better neighborhood solution is found")
        break
    iter = iter + 1

-------------------------------------
iter:  1
no better neighborhood solution is found
